In [10]:
import unittest
import warnings
import numpy as np
import logging as log
import subprocess as sp
import random as random
import parmed as pmd
import paprika
from paprika.openmm_simulate import *
from paprika.restraints import *
from paprika.utils import HAS_OPENMM, decompose_openmm_energy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
my_simulation = OpenMM_GB_simulation()
my_simulation.topology = '../test/cb6-but/vac.topo'
my_simulation.md['temperature'] = 300
my_simulation.md['nonbonded_method'] = app.NoCutoff
my_simulation.md['friction'] = 1/unit.picosecond
my_simulation.md['timestep'] = 0.0005*unit.picoseconds
my_simulation.md['constraints'] = None
my_simulation.md['platform'] = 'Reference'
my_simulation.md['precision'] = 'double'
my_simulation.md['coordinates'] = '../test/cb6-but/vac.crds'
my_simulation.md['steps'] = 10
my_simulation.md['output'] = 'tmp.nc'

In [14]:
rest1 = DAT_restraint()
rest1.amber = True
rest1.continuous_apr = False
rest1.auto_apr = False
rest1.topology = './cb6-but/cb6-but-notcentered.pdb'
rest1.mask1 = ':CB6@O,O2,O4,O6,O8,O10'
rest1.mask2 = ':BUT@C3'
rest1.attach['target'] = 3.0
rest1.attach['num_windows'] = 4
rest1.attach['fc_initial'] = 0.0
rest1.attach['fc_final'] = 3.0
rest1.pull['fc'] = rest1.attach['fc_final']
rest1.pull['num_windows'] = 4
rest1.pull['target_initial'] = rest1.attach['target']
rest1.pull['target_final'] = 6.0
rest1.release['target'] = rest1.pull['target_final']
rest1.release['num_windows'] = rest1.attach['num_windows']
rest1.release['fc_initial'] = rest1.attach['fc_initial']
rest1.release['fc_final'] = rest1.attach['fc_final']
rest1.initialize()

2018-02-06 11:42:35 AM Calculating attach targets and force constants...
2018-02-06 11:42:35 AM Attach, Method #1
2018-02-06 11:42:35 AM Calculating pull targets and force constants...
2018-02-06 11:42:35 AM Pull, Method #1
2018-02-06 11:42:35 AM Calculating release targets and force constants...
2018-02-06 11:42:35 AM Release, Method #1
2018-02-06 11:42:35 AM Number of attach windows = 4
2018-02-06 11:42:35 AM Number of pull windows = 4
2018-02-06 11:42:35 AM Number of release windows = 4
2018-02-06 11:42:35 AM Assigning atom indices...
2018-02-06 11:42:35 AM Loaded ./cb6-but/cb6-but-notcentered.pdb...
2018-02-06 11:42:35 AM There are 6 atoms in the mask :CB6@O,O2,O4,O6,O8,O10  ...
2018-02-06 11:42:35 AM Loaded ./cb6-but/cb6-but-notcentered.pdb...
2018-02-06 11:42:35 AM There are 1 atoms in the mask :BUT@C3  ...


In [142]:
forces = []
for i in [0, 1]:
    prmtop = app.AmberPrmtopFile('../test/cb6-but/vac.topo')
    inpcrd = app.AmberInpcrdFile('../test/cb6-but/vac.crds')
#     system = prmtop.createSystem(nonbondedMethod=app.NoCutoff, constraints=None)
#     integrator = mm.LangevinIntegrator(300, 1/unit.picosecond,  0.0005*unit.picoseconds)
#     simulation = app.Simulation(prmtop.topology, system, integrator, mm.Platform.getPlatformByName('Reference'))
    
    system = my_simulation.setup_system(my_simulation.md, seed=42)    
    
    if i:
        print('Adding restraints...')
        bond_restraint = mm.CustomBondForce('k * (r - r_0)^2')
        bond_restraint.addPerBondParameter('k')
        bond_restraint.addPerBondParameter('r_0')

        r_0 = 10.5 * unit.nanometers
        k = 1000 * unit.kilojoules_per_mole / unit.nanometers ** 2
        bond_restraint.addBond(1, 100, [k, r_0])
        system.addForce(bond_restraint)

    simulation = my_simulation.setup_simulation(system, my_simulation.md)
        
    # simulation.context.setPositions(inpcrd.positions)
    simulation.context.setVelocitiesToTemperature(300 * unit.kelvin, 42)
    simulation.step(1000)
    
    omm_f = simulation.context.getState(getForces=True).getForces(asNumpy=True).value_in_unit(unit.kilocalories_per_mole/unit.angstroms)
    forces.append(omm_f)

    state = simulation.context.getState(getEnergy=True)
    energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
    print(energy)

[autoreload of paprika.restraints failed: Traceback (most recent call last):
  File "/home/dslochower/data/applications/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: __init__() requires a code object with 1 free vars, not 0
]
2018-02-01 08:17:33 PM Integrator random number seed: 42
2018-02-01 08:17:33 PM Platform: Reference
2018-02-01 08:17:44 PM Integrator random number seed: 42
2018-02-01 08:17:44 PM Platform: Reference


-729.419987123009
Adding restraints...
22847.43843452656


In [61]:
simulation.system

<simtk.openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7fe633b751b0> >

In [3]:
for restraint_on in [0, 1]:
    energies = []
    prmtop = app.AmberPrmtopFile('../test/cb6-but/vac.topo')
    inpcrd = app.AmberInpcrdFile('../test/cb6-but/vac.crds')
    system = prmtop.createSystem(nonbondedMethod=app.NoCutoff, constraints=None)
    integrator = mm.LangevinIntegrator(300, 1/unit.picosecond,  0.0005*unit.picoseconds)

    for force in system.getForces():
        force.setForceGroup(10)
    
    if restraint_on:
        bond_restraint = mm.CustomBondForce('k * (r - r_0)^2')
        bond_restraint.addPerBondParameter('k')
        bond_restraint.addPerBondParameter('r_0')

        r_0 = 10.5 * unit.nanometers
        k = 1000 * unit.kilojoules_per_mole / unit.nanometers ** 2
        bond_restraint.addBond(1, 100, [k, r_0])
        system.addForce(bond_restraint)
        bond_restraint.setForceGroup(1)

    simulation = app.Simulation(prmtop.topology, system, integrator, mm.Platform.getPlatformByName('Reference'))   

    simulation.context.setPositions(inpcrd.positions)
    simulation.context.setVelocitiesToTemperature(300 * unit.kelvin, 42)
    simulation.step(10)
    
    total_state = simulation.context.getState(getEnergy=True)
    total_energy = total_state.getPotentialEnergy() / unit.kilocalories_per_mole
    
    if restraint_on:
        restraint_state = simulation.context.getState(getEnergy=True, groups={1})
        restraint_energy = restraint_state.getPotentialEnergy() / unit.kilocalories_per_mole
        
        non_restraint_state = simulation.context.getState(getEnergy=True, groups={10})
        non_restraint_energy = non_restraint_state.getPotentialEnergy() / unit.kilocalorie_per_mole
    
    print('Total energy = {0:4.4f}'.format(total_energy))
    if restraint_on:
        print('Restraint energy = {0:4.4f}'.format(restraint_energy))
        print('Non-restraint energy = {0:4.4f}'.format(non_restraint_energy))

Total energy = -641.2481
Total energy = 23353.1320
Restraint energy = 23931.3301
Non-restraint energy = -578.1982


In [17]:
system = my_simulation.setup_system(my_simulation.md, seed=42)    

for force in system.getForces():
    force.setForceGroup(10)

    
system = my_simulation.add_openmm_restraints(system, [rest1], phase='attach', window=3)
    
simulation = my_simulation.setup_simulation(system, my_simulation.md)
simulation.step(10)

total_state = simulation.context.getState(getEnergy=True)
total_energy = total_state.getPotentialEnergy() / unit.kilocalories_per_mole

restraint_state = simulation.context.getState(getEnergy=True, groups={1})
restraint_energy = restraint_state.getPotentialEnergy() / unit.kilocalories_per_mole

non_restraint_state = simulation.context.getState(getEnergy=True, groups={10})
non_restraint_energy = non_restraint_state.getPotentialEnergy() / unit.kilocalorie_per_mole

print('Total energy = {0:4.4f}'.format(total_energy))
print('Restraint energy = {0:4.4f}'.format(restraint_energy))
print('Non-restraint energy = {0:4.4f}'.format(non_restraint_energy))

2018-02-06 11:43:09 AM Integrator random number seed: 42
2018-02-06 11:43:09 AM Added bond restraint between :CB6@O,O2,O4,O6,O8,O10 and :BUT@C3 with target value = 3.0 A and force constant = 3.0 kcal/(A**2 mol)
2018-02-06 11:43:09 AM Platform: Reference


Total energy = -799.7242
Restraint energy = 15.5837
Non-restraint energy = -815.3078
